In [1]:
import requests
import time
import os
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO

In [2]:
# !pip install duckduckgo-images-api
from duckduckgo_images_api import search

In [ ]:
# Test if images make sense
results = search("search")

img_urls = [r['image'] for r in results['results']]

f, axes = plt.subplots(2, 2, figsize=(15, 15))
for i in range(2):
    for j in range(2):
        img_data = requests.get(img_urls[:4][i+2*j])
        img_data.raise_for_status()
        img = Image.open(BytesIO(img_data.content))        
        axes[i][j].imshow(img)
        axes[i][j].axis("off")
plt.show()

In [19]:
def export_imgs(search_terms, n_train_imgs, size):
    
    imgs_added = set() # keep this to minimize duplicates
    train_i = 0 # keep count to control images that get added to train and test sets
    test_i = 0
    
    for st in range(len(search_terms)):

        # Get image URLs for search_term (default is 100 images)
        results = search(search_terms[st])
        img_urls = [r['image'] for r in results['results']]


        # Clean up image urls and add to respective folder 
        for url in img_urls:
            
            # After n_train_imgs images have been added to training set, add the remaining images to the test set
            if train_i >= n_train_imgs*(st+1):
                is_train = False
            else:
                is_train = True
                
            if is_train:
                # Create path to store images
                dir_path = f"./train/"
                if not os.path.exists(dir_path):
                    os.makedirs(dir_path)
            else:
                dir_path = f"./test/"
                if not os.path.exists(dir_path):
                    os.makedirs(dir_path)

            ## this will check if there is a '?' or '!' after the '.jpg' and get rid of them
            split = url.split("/")

            last_item = split[-1]

            second_split = last_item.split("?")

            if len(second_split) > 1:
                last_item = second_split[0]

            third_split = last_item.split("!")

            if len(third_split) > 1:
                last_item = third_split[0]      
            ##

            # Save images with resize
            if is_train:
                path = os.path.join(dir_path, f'{train_i}.jpg')
            else:
                path = os.path.join(dir_path, f'{test_i}.jpg')
            if last_item not in imgs_added:
                try:
                    img_data = requests.get(url)
                    #image_data.raise_for_status()
                    img = Image.open(BytesIO(img_data.content))
                    img_resized = img.resize(size)
                    img_resized.save(path)
                    if is_train:
                        train_i += 1 
                    else:
                        test_i += 1 # keep track of images added
                except OSError:
                    pass
            imgs_added.add(last_item)
            
            
            # Save images without resize
#             path = os.path.join(dir_path, f'{i}.jpg')
#             if last_item not in imgs_added:
#                 try:
#                     with open(path, "wb") as f:
#                         img_data = requests.get(url)
#                         #image_data.raise_for_status()

#                         f.write(img_data.content)
#                         i += 1 # keep track of images added
#                 except OSError:
#                     pass
#             imgs_added.add(last_item)

In [ ]:
qbs_terms = ['tom brady', 'patrick mahomes']
n_train = 60
resize_size = (600, 400)

export_imgs(qbs_terms, n_train, resize_size)

In [ ]:
# Using makesense.ai labelling tool:
    # clean images - delete duplicates, label all faces and label 'other' if not a qb + draw bounding boxes
# create dataset with all labels (qb + bounding box info)